In [3]:

! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.0 environment at: C:\Projects\LLMOPS\.venv
Resolved 75 packages in 1.55s
 Downloaded shapely
 Downloaded pydantic-core
 Downloaded sqlalchemy
 Downloaded pillow
 Downloaded openai
 Downloaded numpy
 Downloaded langchain-community
 Downloaded rapidocr-onnxruntime
 Downloaded onnxruntime
 Downloaded sympy
 Downloaded opencv-python
Prepared 71 packages in 13.92s
Installed 71 packages in 5.29s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.12.0
 + attrs==25.4.0
 + certifi==2025.11.12
 + charset-normalizer==3.4.4
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + distro==1.9.0
 + flatbuffers==25.12.19
 + frozenlist==1.8.0
 + greenlet==3.3.0
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.3
 + humanfriendly==10.0
 + idna==3.11
 + jiter==0.12.0
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + langchain==1.2.0
 + langchain-classic==1.0.1
 + langchain-community==0.4.1
 + langchain-core==1.2.5
 + langcha

In [4]:
import os

from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [13]:
!uv pip install langchain_community

Using Python 3.12.0 environment at: C:\Projects\LLMOPS\.venv
Resolved 46 packages in 84ms
 Downloaded numpy
Prepared 1 package in 4.91s
Installed 20 packages in 2.04s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + attrs==25.4.0
 + dataclasses-json==0.6.7
 + frozenlist==1.8.0
 + greenlet==3.3.0
 + httpx-sse==0.4.3
 + langchain-classic==1.0.1
 + langchain-community==0.4.1
 + langchain-text-splitters==1.1.0
 + marshmallow==3.26.2
 + multidict==6.7.0
 + mypy-extensions==1.1.0
 + numpy==2.4.0
 + propcache==0.4.1
 + pydantic-settings==2.12.0
 + sqlalchemy==2.0.45
 + typing-inspect==0.9.0
 + yarl==1.22.0


In [14]:
from langchain_community.document_loaders import TextLoader

In [17]:
loader = TextLoader("sample.txt", encoding="utf8")
documents = loader.load()

In [18]:
len(documents)

1

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:

text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

In [25]:
text_chunks=text_splitter.split_documents(documents)

In [26]:
len(text_chunks)

500

In [36]:
! uv pip install faiss-cpu langchain_openai langchain_core

Using Python 3.12.0 environment at: C:\Projects\LLMOPS\.venv
Resolved 36 packages in 687ms
Prepared 1 package in 223ms
Installed 8 packages in 671ms
 + distro==1.9.0
 + jiter==0.12.0
 + langchain-openai==1.1.6
 + openai==2.14.0
 + regex==2025.11.3
 + sniffio==1.3.1
 + tiktoken==0.12.0
 + tqdm==4.67.1


In [37]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


In [38]:
embeddings=OpenAIEmbeddings()

In [39]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [40]:
vectorstore

In [41]:
retriever=vectorstore.as_retriever()

In [42]:
query = "Who is Dhoni"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)

Document 1:
Mahendra Singh Dhoni ([məˈɦeːnd̪ɾə ˈsɪŋɡʱ ˈd̪ʱoːniː] ⓘ; born 7 July 1981) is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the
--------------------------------------------------
Document 2:
Dhoni with Chennai Super Kings in 2011
--------------------------------------------------
Document 3:
"Dhoni named captain of ICC ODI team". The Hindu. 3 December 2013. ISSN 0971-751X. Archived from the original on 27 October 2021. Retrieved 27 October 2021.
--------------------------------------------------
Document 4:
Dhoni in 2008
--------------------------------------------------


In [44]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [45]:
prompt=ChatPromptTemplate.from_template(template)

In [46]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [49]:
from langchain_core.output_parsers import StrOutputParser

In [50]:
output_parser=StrOutputParser()

In [52]:
from langchain_openai import ChatOpenAI

llm_model=ChatOpenAI(model_name="gpt-4o-mini")

In [56]:
from langchain_core.runnables import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [57]:
rag_chain.invoke("tell me about Dhoni")

"Mahendra Singh Dhoni, born on July 7, 1981, is an Indian professional cricketer known for his role as a right-handed batter and wicket-keeper. He is widely regarded as one of the greatest captains in cricket history. Dhoni led the Indian national team to significant successes, including winning the ICC T20 World Cup in 2007 and the ICC Cricket World Cup in 2011. He has played for the Chennai Super Kings in the Indian Premier League (IPL), being a pivotal figure since the tournament's inception in 2008. Dhoni is also noted for his calm demeanor under pressure and exceptional leadership skills. His finishing ability in matches has made him a legendary figure in the sport."